# RTK Data Extraction

**Requirements:**
- RTKLib must be installed
- Run the first 3 cells and fill data into the created directories before starting

**Known Issues:**
- Rolls around 1200 UTC may get split across files.
- Timestamps are displayed in UTC+4

In [ ]:
### Constants
### This should be the only part of the file that needs editing

BUGGIES = ["rover"]

# YYYY-MM-DD
DATE = "YYYY-MM-DD"

CLEANUP = False
DEBUG = False


## Pipe 0: Setup

This will create all the directories needed for analysis. Please make sure to fill out the buggies and dates above.

In [ ]:
### Imports
import os
import csv
import datetime as dt
import math

from geopy.distance import geodesic
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly
from pyubx2 import UBXReader
import seaborn as sns
from scipy.interpolate import Akima1DInterpolator, CubicSpline, splrep, splprep, splev, BSpline
import subprocess
from tqdm import tqdm

from constants import *
from world import *


In [ ]:
# TODO: Firgure out what these args do and utilize them if helpful
rnx2rtkpArgs = ["-m 5"] #, "-i -d 9 -v 2", "-d 9 -v 2"]

BASESTATION_NAME = "basestation"
ROOT_DATA_DIR = "rolls_data"
BASESTATION_DIR = os.path.join(ROOT_DATA_DIR, f"{BASESTATION_NAME}_data")

RAW_DATA_DIR = "raw_ubx"
DECODE_DIR = ".decoded_ubx"
CSV_DIR = ".csv"
PROCESSED_CSV_DIR = "calc_csv"
GRAPHS_DIR = "graphs"

if not os.path.exists(ROOT_DATA_DIR):
    os.mkdir(ROOT_DATA_DIR)
    os.mkdir(BASESTATION_DIR)

BASESTATION_DATE_DIR = os.path.join(BASESTATION_DIR, DATE)
if not os.path.exists(BASESTATION_DATE_DIR):
    os.mkdir(BASESTATION_DATE_DIR)

buggy_dirs = []
for buggy in BUGGIES:
    buggy_dir = os.path.join(ROOT_DATA_DIR, buggy.lower())
    if not os.path.exists(buggy_dir):
        os.mkdir(buggy_dir)

    roll_dir = os.path.join(buggy_dir, DATE)
    if not os.path.exists(roll_dir):
        os.mkdir(roll_dir)

    intake = os.path.join(roll_dir, RAW_DATA_DIR)
    if not os.path.exists(intake): 
        os.mkdir(intake)

    decode = os.path.join(roll_dir, DECODE_DIR)
    if not os.path.exists(decode): os.mkdir(decode)

    raw_csv = os.path.join(roll_dir, CSV_DIR)
    if not os.path.exists(raw_csv): os.mkdir(raw_csv)

    processed_csv = os.path.join(roll_dir, PROCESSED_CSV_DIR)
    if not os.path.exists(processed_csv): os.mkdir(processed_csv)

    graphs = os.path.join(roll_dir, GRAPHS_DIR)
    if not os.path.exists(graphs): os.mkdir(graphs)

    buggy_dirs.append(roll_dir)

In [ ]:
needs_data = False

if not os.listdir(BASESTATION_DATE_DIR):
    print(f"Please fill {BASESTATION_DATE_DIR} with basestation data!")
    needs_data = True

for roll_dir in buggy_dirs:
    intake = os.path.join(roll_dir, RAW_DATA_DIR)
    if not os.listdir(intake): 
        print(f"Please fill {intake} with roll data!")
        needs_data = True

# Little check to stop the program if data is missing
# Continue after adding data
if needs_data:
    raise FileNotFoundError

In [ ]:
for rover_dir in buggy_dirs:
    intake_dir = os.path.join(rover_dir, RAW_DATA_DIR)

    # Make sure everything has a .ubx extension (for convbin)
    bin_files = [os.path.join(intake_dir, f) for f in os.listdir(intake_dir) if f.endswith(".BIN")]
    for file in bin_files:
        prefix, _ = os.path.splitext(file)
        new_name = prefix + ".ubx"
        os.rename(src=file, dst=new_name)
    
    rover_ubx = sorted([f for f in os.listdir(intake_dir) if f.endswith(".ubx")])
    if DEBUG: print(rover_dir, ":", rover_ubx)

## Pipe 1: Postprocessing

`.ubx` --> `.pos`, `.pos.stat`, and `_events.pos` files.

This notebook contains scripts for obtaining RTK solutions through various configurations and validating/assessing the quality of those solutions.

`convbin`:
Input files are `base.ubx` and `rover.ubx`.
Generates `.obs`, `.nav`, and `.sbs` files.

`rnx2rtkp`:
Input files are `rover.obs`, `base.obs`, and `rover.nav`.
Generates `.pos`, `.pos.stat`, and `_events.pos` files.

In [ ]:
basestation_files = [file for file in os.listdir(BASESTATION_DATE_DIR) if file.endswith(".ubx")]
if not basestation_files:
    # Remember to add and/or unzip the basestation files!
    raise FileNotFoundError
    
for i, base_file in enumerate(basestation_files):
    date, time, ext = base_file.split('_')
    
    if date != DATE:
        print("Found file with wrong date:", base_file)
        continue

    if DEBUG: print(base_file)
    
    base_full_path = os.path.join(BASESTATION_DATE_DIR, base_file)
    base_obs = os.path.join(BASESTATION_DATE_DIR, f"{BASESTATION_NAME}-{time}.obs")
    base_nav = os.path.join(BASESTATION_DATE_DIR, f"{BASESTATION_NAME}-{time}.nav")
    base_sbs = os.path.join(BASESTATION_DATE_DIR, f"{BASESTATION_NAME}-{time}.sbs")

    if os.path.exists(base_obs):
        print("obs file", base_obs, "already exists. Skipping")
        continue

    subprocess.run(["convbin", base_full_path, "-o", base_obs, "-n", base_nav, "-s", base_sbs])
    # os.system(f"convbin {base_full_path} -o {base_obs} -n {base_nav} -s {base_sbs}")

    # Sanity check files are good
    if os.path.getsize(base_nav) < 1000 or os.path.getsize(base_obs) < 10000:
        raise Exception(f"Base station file {base_full_path} is borked.")

In [ ]:
valid_rolls = []

for rover_dir in buggy_dirs:
    intake_dir = os.path.join(rover_dir, RAW_DATA_DIR)
    decode_dir = os.path.join(rover_dir, DECODE_DIR)
    rover_ubx = sorted([f for f in os.listdir(intake_dir) if f.endswith(".ubx")])

    for file in rover_ubx:
        if DEBUG: print(rover_dir, file)
        roll_no = ''.join([n for n in os.path.basename(file) if n.isdigit()])
        obs = os.path.join(decode_dir, f"{roll_no:04}.obs")
        nav = os.path.join(decode_dir, f"{roll_no:04}.nav")
        sbs = os.path.join(decode_dir, f"{roll_no:04}.sbs")

        if os.path.exists(obs):
            print("obs file", obs, "already exists. Skipping")
            valid_rolls.append((rover_dir, obs, nav))
            continue

        raw_ubx_path = os.path.join(intake_dir, file)
        # os.system(f"convbin {file} -o {obs} -n {nav} -s {sbs}")
        subprocess.run(["convbin", raw_ubx_path, "-o", obs, "-n", nav, "-s", sbs])

        # Sanity check files are good
        if not os.path.exists(obs):
            print(f"Files not created from {raw_ubx_path}")
            continue

        obs_size = os.path.getsize(obs)
        nav_size = os.path.getsize(nav)
        if obs_size < 5000 or nav_size < 1000:
            print("Obs Size:", obs_size)
            print("Nav Size:", nav_size)
            raise Exception(f"convbin borked. Originating from {raw_ubx_path}")

        valid_rolls.append((rover_dir, roll_no))
if DEBUG: print(valid_rolls)


In [ ]:
good_pos_files = []

for roll in valid_rolls:
    if DEBUG: print(roll)
    obs = os.path.join(roll[0], DECODE_DIR, roll[1] + ".obs")
    nav = os.path.join(roll[0], DECODE_DIR, roll[1] + ".nav")

    for i, base_obs in enumerate(sorted([os.path.join(BASESTATION_DATE_DIR, f) for f in os.listdir(BASESTATION_DATE_DIR) if f.endswith(".obs")])):
        for j, arg in enumerate(rnx2rtkpArgs):
            output_file = f"{roll[1]:04}-{i}-{j}.pos"
            out_file_path = os.path.join(roll[0], DECODE_DIR, output_file)
            os.system(f"rnx2rtkp -o {out_file_path} {arg} -p 2 -y 2 -sys G,R,E,C -t -s , -l 40.442403483 -79.946996633 286.257 {obs} {base_obs} {nav}")
            
            if os.path.exists(out_file_path) and os.path.getsize(out_file_path) > 2000:
                print(f"Created output file {output_file}")
                good_pos_files.append((roll[0], output_file))

# Pipe 2: Parsing

This notebook parses `.pos` files output by the previous pipe, and generates `.csv` files with headers.
**No math is involved.**


In [ ]:
def posToCsv(inputPath, outputPath):
    with open(inputPath, 'r') as file:
        lines = [line for line in file.readlines() if not line.startswith("%")] # removes lines beginning with %
        lines = [line.strip() for line in lines] # remove leading and trailing whitespace from each line
        lines = [[value.strip() for value in line.split(",")] for line in lines] # removes leading and trailing whitespace from each CSV value in each line

    rows = []
    for line in lines:
        row = {}
        for i in range(len(line)):
            row[PARSE_CSV_HEADER[i]] = line[i]
        rows.append(row)
    
    with open(outputPath, 'w') as file:
        writer = csv.DictWriter(file, fieldnames=PARSE_CSV_HEADER)
        writer.writeheader()
        writer.writerows(rows)


In [ ]:
if DEBUG: print(good_pos_files)

for dir, file in good_pos_files:
    input_path = os.path.join(dir, DECODE_DIR, file)

    prefix, _ = os.path.splitext(file)
    buggy = os.path.basename(os.path.dirname(dir))
    new_name = '_'.join([buggy, DATE, prefix]) + ".csv"
    output_path = os.path.join(dir, CSV_DIR, new_name)
    
    posToCsv(input_path, output_path)

# Pipe 3: Analysis

This notebook contains scripts for calculating kinematic data.

Input files must be `.csv` files generated by `rtklib-pos-to-csv.ipynb`.
Its headers are defined in the code below.

Outputs another `.csv` file _with the **different** headers_.
Those new headers are also defined in the code below.

In [ ]:
TIMESTAMP_FORMAT = "%Y/%m/%d %H:%M:%S.%f"
TIME_OFFSET = dt.timedelta(hours=4)

# TODO make this compatible with several levels of precesion
# converts timestamp string to UTC value (float)

def timestampToLocal(ts: str):
    return (dt.datetime.strptime(ts, TIMESTAMP_FORMAT) - TIME_OFFSET).astimezone().strftime("%c")


def timestampToValue(ts: str):
    return (dt.datetime.strptime(ts, TIMESTAMP_FORMAT) - TIME_OFFSET).timestamp()


In [ ]:
def getMagnitude(vector):
    return math.sqrt(math.pow(vector[0], 2) + math.pow(vector[1], 2))

# Conversion factor from m/s to mph
FREEDOM_MULTIPLIER = 2.236936

In [ ]:
"""
Code yoinked from https://github.com/CMU-Robotics-Club/RoboBuggy2/blob/sw/20230331/rb_ws/src/buggy/scripts/auton/world.py

    Abstraction for the world coordinate system

    The real world uses GPS coordinates, aka latitude and longitude. However,
    using lat/lon is bad for path planning for several reasons. First, the difference
    in numbers would be very tiny for small distances, alluding to roundoff errors.
    Additionally, lat/lon follows a North-East-Down coordinate system, with headings
    in the clockwise direction. We want to use an East-North-Up coordinate system, so
    that the heading is in the counter-clockwise direction.

    We do this by converting GPS coordinates to UTM coordinates, which are in meters.
    UTM works by dividing the world into a grid, where each grid cell has a unique
    identifier. A UTM coordinate consists of the grid cell identifier and the "easting"
    and "northing" within that grid cell. The easting (x) and northing (y) are in meters,
    and are relative to the southwest corner of the grid cell.

    Last, we offset the UTM coordinates to be relative to some arbitrary zero point. That
    way, the final world coordinates are relatively close to zero, which makes debugging
    easier.

    This class provides methods to convert between GPS and world coordinates. There is
    a version for single coordinates and a version for numpy arrays.
"""

def gps_to_world(lat, lon):
    """Converts GPS coordinates to world coordinates

    Args:
        lat (float): latitude
        lon (float): longitude

    Returns:
        tuple: (x, y) in meters from some arbitrary zero point
    """
    utm_coords = utm.from_latlon(lat, lon)
    x = utm_coords[0] - World.WORLD_EAST_ZERO
    y = utm_coords[1] - World.WORLD_NORTH_ZERO

    return x, y

In [ ]:
def calc(inputPath, outputPath):
    input_df = pd.read_csv(inputPath)

    # Converting to linear local coordinatess
    local_coords = []
    for i in range(len(input_df)):
        local_coords.append(gps_to_world(input_df["latitude"][i], input_df["longitude"][i]))

    input_df["latitude_meters"] = [coord[0] for coord in local_coords]
    input_df["longitude_meters"] = [coord[1] for coord in local_coords]


    interpolator = Akima1DInterpolator(input_df.loc[:, "timestamp"].map(timestampToValue), input_df.loc[:, ["latitude_meters", "longitude_meters"]])
    # interpolator = CubicSpline(input_df.loc[:, "timestamp"].map(timestampToValue), input_df.loc[:, ["latitude", "longitude"]])
    positions = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=0)
    velocities = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=1)
    accelerations = interpolator(input_df.loc[:, "timestamp"].map(timestampToValue), nu=2)

    output_df = pd.DataFrame(columns=CALC_CSV_HEADER)

    output_df["timestamp"] = input_df.loc[:, "timestamp"].map(timestampToValue)
    output_df["timestamp_str"] = input_df["timestamp"].map(timestampToLocal)

    output_df["fix_type"] = input_df["fix-type"]

    output_df["latitude"] = input_df["latitude"]
    output_df["longitude"] = input_df["longitude"]
    output_df["altitude"] = input_df["altitude"]

    output_df["velocity"] = velocities.tolist()
    output_df["speed"] = [getMagnitude(vector) * FREEDOM_MULTIPLIER for vector in velocities]

    output_df["acceleration_vector"] = accelerations.tolist()
    output_df["acceleration_magnitude"] = [getMagnitude(vector) for vector in accelerations]

    output_df = output_df[output_df["speed"] < 44] # Filter out anything faster than 50mph

    output_df.to_csv(outputPath)

In [ ]:
for rover_dir in buggy_dirs:
    raw_csv_dir = os.path.join(rover_dir, CSV_DIR)
    clean_csv_files = sorted([f for f in os.listdir(raw_csv_dir) if f.endswith(".csv")])
    if DEBUG: print(clean_csv_files)

    for name in clean_csv_files:
        input_path = os.path.join(rover_dir, CSV_DIR, name)
        output_path = os.path.join(rover_dir, PROCESSED_CSV_DIR, name)
        calc(input_path, output_path)

# Pipe 4: Visualization

Notebook contains scripts for visualizing kinematic data.

Inputs files must be `.csv` file output by `analysis.ipynb`.

_Note that these `.csv` files are different from those generated by `rtklib-pos-to-csv.ipynb` as they have different headers._

In [ ]:
def plot_map(df, outputFilepath):
    layers = [{"below": "traces",
               "sourcetype": "raster",
               "source": ["https://imagery.pasda.psu.edu/arcgis/rest/services/pasda/PEMAImagery2018_2020/MapServer/WMTS/tile/1.0.0/pasda_PEMAImagery2018_2020/default/default028mm/{z}/{y}/{x}.png"]}]

    fig = px.scatter_mapbox(df,
                            lat="latitude", lon="longitude",
                            color="speed",
                            hover_data=["altitude",
                                        "acceleration_magnitude",
                                        "timestamp_str"],
                            zoom=15, size_max=18,
                            height=400, width=800)

    fig.update_layout(
        mapbox_style="white-bg",
        mapbox_layers=layers
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    # fig.show()
    plotly.offline.plot(fig, filename=outputFilepath)

In [ ]:
def plot_speed_graph(df, outputFilepath):
    plt.figure().set_figwidth(15)
    plt.plot(df["timestamp"], df["speed"])
    plt.savefig(outputFilepath)

def plot_accel_graph(df, outputFilepath):
    plt.figure().set_figwidth(15)
    plt.plot(df["timestamp"], df["acceleration_magnitude"])
    plt.savefig(outputFilepath)

def plot_speed_accel_graph(df, outputFilepath):
    fig, (speed, accel) = plt.subplots(2)
    fig.set_figwidth(15)
    speed.plot(df["timestamp"], df["speed"])
    speed.set_ylim([0, 20])
    accel.plot(df["timestamp"], df["acceleration_magnitude"])
    accel.set_ylim([-1, 50])
    plt.savefig(outputFilepath)

In [ ]:
for rover_dir in buggy_dirs:
    calc_csv_dir = os.path.join(rover_dir, PROCESSED_CSV_DIR)
    calc_files = sorted([f for f in os.listdir(calc_csv_dir) if f.endswith(".csv")])

    for file in calc_files:
        df = pd.read_csv(os.path.join(calc_csv_dir, file))
        roll_name, _ = os.path.splitext(file)
        output_path = os.path.join(rover_dir, GRAPHS_DIR)
        plot_map(df, os.path.join(output_path, f"{roll_name}-map.html"))
        # plot_speed_graph(df, os.path.join(output_path, f"{roll_name}-speed.png"))
        # plot_accel_graph(df, os.path.join(output_path, f"{roll_name}-accel.png"))
        # plot_speed_accel_graph(df, os.path.join(output_path, f"{roll_name}-speed-accel.png"))

# Pipe 5: Cleanup

If `CLEANUP` is set to true, all files except for the plots will be deleted.

In [ ]:
if CLEANUP:
    for buggy in BUGGIES:
        buggy_dir = os.path.join(ROOT_DATA_DIR, buggy.lower())
        roll_dir = os.path.join(buggy_dir, DATE)
        
        decode = os.path.join(roll_dir, DECODE_DIR)
        if os.path.exists(decode):
            for f in os.listdir(decode):
                os.remove(os.path.join(decode, f))
            os.rmdir(decode)

        raw_csv = os.path.join(roll_dir, CSV_DIR)
        if os.path.exists(raw_csv): 
            for f in os.listdir(raw_csv):
                os.remove(os.path.join(raw_csv, f))
            os.rmdir(raw_csv)
        
        